In [12]:
using Plots, GLM, RDatasets
;

In [2]:
function holding_cost(x,y,factor) 
    return -1*factor*(div(x,y)+1)
end

function shipping_cost(x,y)
    shipping_factor = 1
    return -1*shipping_factor*(div(x,y)+1)
end

function pre_process_cost(x,t,hFactor,P0)
    ## Cost to materials, and normalized by weight
    # [$/lb]
    m = -0.07 # manual demanufacturing 
    s = -0.04 # shredding 
    f = -0.03 # metal finder 

    ## Price to sell for TV's
    # price per pound multiplied by percentage of TV unit is
    stl     = 0.06 * 0.0299 
    brds2   = 0.35 * 0.0699
    wre      = 0.46 * 0.0099
    ptc   = 0.07 * 0.0599
    cne     = 0.42 * 0.0249
    dwr     = 0.65 * 0.0099
    pcpr   = 0.08 * 0.0240
    palm     = 0.45 * 0.0019
    tube      = 0.06 * 0.1499
    ldgls = -0.13 * 0.62
    selling_price = sum([stl brds2 wre ptc cne dwr pcpr palm tube ldgls])
    
    cost = x*(P0+m+s+f+selling_price) + 
            t*holding_cost(x,451,hFactor) + shipping_cost(x,21600)
    return  cost
end

function post_process_cost(x, proccess_t, t,hFactor,P0)
    orig_weight = x/0.62
    accum_cost = pre_process_cost(orig_weight, proccess_t, hFactor,P0)
    
    return t*holding_cost(x,451,hFactor) + accum_cost
end
;

In [3]:
price=0.05:0.05:1.5
n = length(price)

zero_no = zeros(n,1)
zero_full = zeros(n,1)
zero_king = zeros(n,1)

prod = 5

h_no = 5.93
h_full = 1.53
h_king = 1.20

weight = 50*450

for i=1:n
    zero_no[i] = pre_process_cost(weight, prod,  h_no, price[i])/
        (h_no*div(0.62*weight,451)) + prod
    zero_full[i] = pre_process_cost(weight, prod, h_full, price[i])/
        (h_full*div(0.62*weight,451)) + prod
    zero_king[i] = pre_process_cost(weight, prod, h_king, price[i])/
        (h_king*div(0.62*weight,451)) + prod
end


In [16]:
plot(price,zero_no,label="no change",title="Price vs Zero-Time",
    xlab="Price",ylab="Days")
plot!(price,zero_full,label="team change")
plot!(price,zero_king,label="King's change")
scatter!([0.175],[0], label="Minimum price for profit 0.17")

In [5]:
price=0.05:0.05:1.0
n = length(price)

no_cost180 = zeros(n,1)
full_cost180 = zeros(n,1)
king_cost180 = zeros(n,1)

prod = 5

h_no = 5.93
h_full = 1.53
h_king = 1.20

weight = 50*450

for i=1:n
    no_cost180[i] = pre_process_cost(weight, prod,  h_no, price[i]) + 
                    post_process_cost(weight*.62, prod, 180, h_no, price[i])
    full_cost180[i] = pre_process_cost(weight, prod, h_full, price[i]) + 
                      post_process_cost(weight*.62, prod, 180, h_full, price[i])
    king_cost180[i] = pre_process_cost(weight, prod, h_king, price[i]) +  
                      post_process_cost(weight*.62, prod, 180, h_king, price[i])
end

In [6]:
plot(price,no_cost180,label="no change",title="Price vs Profit at 6 months",xlab="Price",ylab="Profit")
plot!(price,full_cost180,label="team change")
plot!(price,king_cost180,label="King's change")
scatter!([0.1],[king_cost180[2]], label="10cent, profit = -9861")
scatter!([.32],[38],label="32cent, profit=38")

In [7]:
price=0.05:0.05:1.0
n = length(price)

no_cost120 = zeros(n,1)
full_cost120 = zeros(n,1)
king_cost120 = zeros(n,1)

prod = 5

h_no = 5.93
h_full = 1.53
h_king = 1.20

weight = 50*450

for i=1:n
    no_cost120[i] = pre_process_cost(weight, prod,  h_no, price[i]) + 
                    post_process_cost(weight*.62, prod, 120, h_no, price[i])
    full_cost120[i] = pre_process_cost(weight, prod, h_full, price[i]) + 
                      post_process_cost(weight*.62, prod, 120, h_full, price[i])
    king_cost120[i] = pre_process_cost(weight, prod, h_king, price[i]) +  
                      post_process_cost(weight*.62, prod, 120, h_king, price[i])
end

In [8]:
plot(price,no_cost120,label="no change",title="Price vs Profit at 4 months",
    xlab="Price",ylab="Profit")
plot!(price,full_cost120,label="team change")
plot!(price,king_cost120,label="King's change")
scatter!([0.1],[king_cost120[2]], label="10cent, profit = -7629")
scatter!([.27],[20],label="27cent, profit=20")